<h1>RadxRate Tutorial</h1>

---

This interactive tutorial takes you through the steps of how to run RadxRate. RadxRate builds upon the capabilities of RadxKdp and RadxPid by calculating hourly rain rates at each radar gate depending on the local PID category and polarimetric values. It shares parameters files with those two applications. RadxRate will calculate the rain rates with either raw or attenuated-corrected data; the user can choose the option they prefer. The three-dimensional rain rates produced by RadxRate can be then used in RadxQpe to estimate the rain rate closest to the surface, after accounting for beam blockage, noise, and clutter.

---

*Note: this tutorial is just one type of workflow to use RadxRate. There are several other workflows!*

<h2>Overview</h2>


**1. Parameter files**
* RadxRate main params
* RadxRate Kdp_specific params
* RadxRate Pid_specific params
* RadxRate Rate_specific params
* Pid Thresholds params (Radar dependent)
* Grib2toMdv params

For a list of the parameters that were changed for this notebook, please refer [here](#Parameter-file-modifications).

https://stackoverflow.com/questions/19699059/print-directory-file-structure-with-icons-for-representation-in-markdown
```
BASE_DIR
│   RadxRate_tutorial.ipynb
│ 
│
└───data
│   │   ...
│   │   
│   │
│   │───raw
│   │   │   KMHX20180914_191822_V06.ar2v
│   │   │   gfsanl_4_2018091418.g2.tar
│   │   │   
│   │───CfRadial
│   │   │   ...
│   │   │
│   │   │
│   │───sounding 
│   │   │   ...
│   │   │
│    ───
│   
└───params
│   │   Pid_thresholds.sband.shv.txt
│   │   RadxRate_main_params
│   │   RadxRate_Kdp_params
│   │   RadxRate_Pid_params
│   │   RadxRate_rate_params



For this tutorial, the parameter files have already been created and populated with the appropriate parameters. However, the default parameter files can be saved using the following commands.

In [ ]:
# # create original parameter files
# !/usr/local/lrose/bin/RadxRate -print_params > /home/jovyan/ams2023/params/user/RadxRate_main_params_user
# !/usr/local/lrose/bin/RadxRate -print_params_kdp > /home/jovyan/ams2023/params/user/RadxRate_Kdp_params_user
# !/usr/local/lrose/bin/RadxRate -print_params_pid > /home/jovyan/ams2023/params/user/RadxRate_Pid_params_user
# !/usr/local/lrose/bin/RadxRate -print_params_rate > /home/jovyan/ams2023/params/user/RadxRate_rate_params_user
# !/usr/local/lrose/bin/Grib2toMdv -print_params > /home/jovyan/ams2023/params/user/Grib2toMdv_params_user

# 1. Set Up
## Set up environment and packages

First, we import the required python packages to run this notebook. Most of the LROSE processing can be done with the os package and shell commands. At the end we will plot the output using Py-ART.

In [ ]:
import os
import pyart
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np

Next, we set up the directory structure to simplify our commands. 

(**note: modify BASE_DIR and LROSE_DIR if your personal machine has different directory structures**)

* BASE_DIR: the base directory containing the directories for the notebooks, data, parameter files
* RADAR_NAME: the name of the radar used in this tutorial
* LROSE_DIR: path to the LROSE installation 

In [ ]:
os.environ['BASE_DIR'] = '/home/jovyan/ams2023'
os.environ['RADAR_NAME'] = 'KMHX'
os.environ['LROSE_DIR'] = '/usr/local/lrose/bin'
!echo "Base directory: "$BASE_DIR
!echo "Base directory: "$RADAR_NAME

## Data download and directory organization

https://drive.google.com/drive/folders/1nQ1-CUR_OFuSdWgsE-qL35L5rcWTmEh4?usp=share_link

We need to set up the required data directories and download the radar data and GFS analysis to the JupyterHub. We delete any existing files and directories specific to this tutorial.

In [ ]:
## make a directory for all the data files in the echo tutorial (raw and analysis)
!rm -rf ${BASE_DIR}/data/echo
!mkdir ${BASE_DIR}/data/echo

## make subdirectory within data for the raw data
!rm -rf ${BASE_DIR}/data/echo/raw
!mkdir ${BASE_DIR}/data/echo/raw

## make subdirectory within data for sounding data
!rm -rf ${BASE_DIR}/data/echo/sounding
!mkdir ${BASE_DIR}/data/echo/sounding

### wget raw files from ftpserver and place in appropriate $BASE_DIR subdirectory
# !wget ftpserver/KMHX20180914_191822_V06.ar2v
# !wget ftpserver/gfsanl_4_2018091418.g2.tar

!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1NT_gA3RPi5PB0PSCh8b8UNm43uc5fQrO' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1NT_gA3RPi5PB0PSCh8b8UNm43uc5fQrO" -O gfsanl_4_2018091418.g2.tar && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=10On5qmT4p-Wvm1kpdGjLUHdu0TQyXZDK' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=10On5qmT4p-Wvm1kpdGjLUHdu0TQyXZDK" -O KMHX20180914_191822_V06.ar2v && rm -rf /tmp/cookies.txt

## move files to proper directories
!mv KMHX20180914_191822_V06.ar2v ${BASE_DIR}/data/echo/raw
!mv gfsanl_4_2018091418.g2.tar ${BASE_DIR}/data/echo/raw


# 2. Prepare data for analysis
## Convert Nexrad Level II to CfRadial files

Now that we have the raw radar data, we convert it into CfRadial using RadxConvert so that the other applications can read the data. Here, the *-f* flag provides the path to the raw radar file and the *-outdir* flag tells RadxConvert where the converted file should be written.

In [ ]:
# !rm -rf ${BASE_DIR}/data/echo/CfRadial
# !mkdir ${BASE_DIR}/data/echo/CfRadial
!${LROSE_DIR}/RadxConvert -f ${BASE_DIR}/data/echo/raw/KMHX20180914_191822_V06.ar2v -outdir ${BASE_DIR}/data/echo/CfRadial


## Untar GFS analysis grib2 files for sounding
* Grb2 --> Mdv --> Spdb

We use the GFS analysis to obtain a sounding for the PID algorithm. We need to untar the files, convert to a gridded file format (Grib2toMdv), then extract the sounding and store in in the Spdb format (Mdv2SoundingSpdb).

In [ ]:
# Grib2Mdv --> Mdv2SoundingSpdb

# make directory for grib files in data/sounding
!rm -rf ${BASE_DIR}/data/echo/sounding/grib
!mkdir ${BASE_DIR}/data/echo/sounding/grib

# extract file from tape archive (tar)
!tar -xvf ${BASE_DIR}/data/echo/raw/gfsanl_4_2018091418.g2.tar -C ${BASE_DIR}/data/echo/sounding/grib/


- **Grib2toMdv**
    - input_dir [line 69] optional, if directory is specified in command line
    - write_non_forecast [line 385]
    - non_forecast_mdv_url (location of outputted mdv files) [line 395]
    - out_projection_info [line 715] Radar location 
    - out_grid_info [line 749] Subjective box around radar(s)
    - height_levels [line 838] Height levels, optional
    
Here, the *-params* flag directs Grib2toMdv to the parameter file and *-f* points to the raw GRIB file.

In [ ]:
# convert model analysis to Mdv
!${LROSE_DIR}/Grib2toMdv -params ${BASE_DIR}/params/echo/Grib2toMdv_params -f ${BASE_DIR}/data/echo/sounding/grib/*.grb2


- **Mdv2SoundingSpdb**
    - input data field names [line 126] 
    - sampling locations [line 264]
        - Need to input location of current radar KMHX
    - ouput_url [line 363]
    
Here, the *-params* flag directs Mdv2SoundingSpdb to the parameter file and *-f* points to the raw GRIB file.

In [ ]:
# convert Mdv to Spdb
!${LROSE_DIR}/Mdv2SoundingSpdb -params ${BASE_DIR}/params/echo/Mdv2SoundingSpdb_params -f ${BASE_DIR}/data/echo/sounding/mdv/20180914/*.mdv*
!${LROSE_DIR}/Mdv2SoundingSpdb -params ${BASE_DIR}/params/echo/Mdv2SoundingSpdb_params -f ${BASE_DIR}/data/echo/sounding/mdv/20180915/*.mdv*


We can check the sounding quality using SpdbQuery, which prints out a summary of the sounding data. Here, *-url* indicates the directory where the Spdb files are located and *-start* and *-end* provide the data range over which we want to inspect the sounding information. 

In [ ]:
!${LROSE_DIR}/SpdbQuery -url ${BASE_DIR}/data/echo/sounding/spdb -start "2018 09 14 00 00 00" -end " 2018 09 14 23 00 00"


# 3. Setup steps complete, run RadxRate

With the radar data and point sounding estimates, we can now run RadxRate. RadxRate calculates KDP, runs the PID algorithm, and estimates the three-dimensional rain rate at each gate. 

Running RadxRate requires one main parameter file on the command line. That parameter file contains  links to the relevant KDP, PID, and rain rate coefficient parameter files. 

Here, *-params* provides the link to the main RadxRate parameter file, *-f* provides the link to the files we want to process, and *-outdir* indicates where RadxRate should write the final files.

In [ ]:
!${LROSE_DIR}/RadxRate -params ${BASE_DIR}/params/echo/RadxRate_main_params -f ${BASE_DIR}/data/echo/CfRadial/20180914/*.nc -outdir ${BASE_DIR}/data/echo/rate


# 4. Plot PID and rate for NEXRAD Morehead City radar (KMHX)

To visualize the output in the notebook, we can use Py-ART.

In [ ]:
# Read CfRadial file into radar object
inDir = "/home/jovyan/ams2023/data/echo/rate/20180914/"
file = "cfrad.20180914_191822.077_to_20180914_192511.621_KMHX_SUR.nc"
rate_kmhx = pyart.io.read_cfradial(inDir+file)
rate_kmhx.info('compact')


We can create a colormap for visualizing PID.

In [ ]:
pidmap = np.array([[0.12156862745098039, 0.46666666666666667, 0.70588235294117652, 1.0],
              [0.68235294117647061, 0.7803921568627451, 0.90980392156862744, 1.0],
              [0.59607843137254901, 0.87450980392156863, 0.54117647058823526, 1.0],
              [0.45490196078431372, 0.7686274509803922, 0.46274509803921571, 1.0],
              [0.17254901960784313, 0.62745098039215685, 0.17254901960784313, 1.0],
              [0.83921568627450982, 0.15294117647058825, 0.15686274509803921, 1.0],
              [1.0, 0.59607843137254901, 0.58823529411764708, 1.0],
              [1.0, 0.49803921568627452, 0.054901960784313725, 1.0],
              [1.0, 0.73333333333333328, 0.47058823529411764, 1.0],
              [0.61960784313725492, 0.85490196078431369, 0.89803921568627454, 1.0],
              [0.090196078431372548, 0.74509803921568629, 0.81176470588235294, 1.0],
              [0.61176470588235299, 0.61960784313725492, 0.87058823529411766, 1.0],
              [0.32156862745098042, 0.32941176470588235, 0.63921568627450975, 1.0]],'f')
my_cmap2 = colors.ListedColormap(pidmap, name='ncar_pid')


In [ ]:
# Plot results of RadxRate

displayRate = pyart.graph.RadarDisplay(rate_kmhx)
figRate = plt.figure(1, (12, 10))

# DBZ (input)

axDbz = figRate.add_subplot(221)
displayRate.plot_ppi('DBZ', 0, vmin=-32, vmax=64.,
                    axislabels=("x(km)", "y(km)"),
                    colorbar_label="DBZ")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# KDP (computed)

axKdp = figRate.add_subplot(222)
displayRate.plot_ppi('KDP', 0, vmin=0, vmax=2.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="KDP (deg/km)",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# RATE_HYBRID (computed)

axHybrid = figRate.add_subplot(223)
displayRate.plot_ppi('RATE_HYBRID', 0, vmin=0, vmax=50.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="RATE_HYBRID(mm/hr)",
    cmap = "pyart_RRate11")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# NCAR PID (computed)

axPID = figRate.add_subplot(224)
displayRate.plot_ppi('PID', 0, vmin=0.5, vmax = 13.5,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="PID",
    cmap = my_cmap2, mask_outside=True)
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

pid_cbar = displayRate.cbs[3]
pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13])
pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice'])
# pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
# pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice', 'slw', 'insects', '2nd-trip', 'clutter'])

figRate.tight_layout()

plt.show()

# 5. Extras

<a id=’Parameter+file+modifications’></a>
## **Parameter file modifications**

Below is a list of the parameters modified for this tutorial. The line numbers are accurate as of Dec 2022 for the final Topaz release of LROSE. 

* **RadxRate main params**
    * input_dir (radar data) [line 105]
    * INPUT FIELD INFORMATION. [line 155]
        * SNR_available = TRUE; change to FALSE; [line 177]
        * DBZ_field_name = "DBZ"; change to "REF"; [line 215]
        * LDR_available = TRUE; change to FALSE [line 255]
    * kdp_params_file_path  [line 284]
    * pid_params_file_path  [line 303]
    * PID_use_attenuation_corrected_fields = TRUE [line 331]
    * RATE_params_file_path [line 350]
    * RATE_use_attenuation_corrected_fields = TRUE [line 378]
    * output_dir [line 699] -- or set in command line with -outdir flag
    * output_format = OUTPUT_FORMAT_CFRADIAL; (Default) [line 755] 
* **RadxRate Kdp_specific params**
    * KDP_fir_filter_len = KDP_FIR_LEN_10; [line 65]
    * KDP_psob_method = PEAK_REMOVAL_METHOD; [line 114]
* **RadxRate Pid_specific params**
    * PID_thresholds_file_path [line 27]
    * PID_use_soundings_from_spdb = FALSE; set to TRUE [line 303]
    * PID_sounding_spdb_url = "spdb/soundings"; [line 313]
    * PID_sounding_location_name [line 339]
    *  **do sounding steps first before filling Pid variables out**
* **RadxRate Rate_specific params**
    * RATE_zh_aa [line 154]
    * RATE_zh_bb [line 164]
    * RATE_zh_aa_snow [line 182]
    * RATE_zh_bb_snow [line 192]
    * RATE_zzdr_aa [line 210]
    * RATE_zzdr_bb [line 220]
    * RATE_zzdr_cc [line 230]
    * RATE_kdp_aa [line 248]
    * RATE_kdp_bb [line 258]
    * RATE_kdpzdr_aa [line 277]
    * RATE_kdpzdr_bb [line 287]
    * RATE_kdpzdr_cc [line 297]
* **Pid Thresholds params**
    * Radar dependent
        * NEXRAD is S-band simultaneous h/v transmission and recieving
        * Other parameter files can be found [here](http://wiki.lrose.net/index.php/RadxPid#PID_thresholds_file)
